In [1]:
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy
from tqdm import tqdm


from model import LR
from data import FairnessDataset, SyntheticDataset, GermanDataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *
from ei_model_dev import FairBatch, Covariance

# warnings.filterwarnings('ignore')

In [2]:
# dataset = SyntheticDataset(seed=0)
dataset = GermanDataset(seed=0)

In [3]:
torch.manual_seed(0)
train_tensors, val_tensors, test_tensors = dataset.tensor(z_blind=False)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

# model_params = torch.load('../fc_erm_model.pkl')
model = LR(train_dataset.X.shape[1])
# model.load_state_dict(model_params)

tau = 0.5
delta = 0.5
# effort_model = Optimal_Effort(delta)
effort_model = PGD_Effort(delta, 50)
ei_model = Covariance(model, effort_model, tau)
ei_model.train(train_dataset, 0.)

for module in model.layers:
    if hasattr(module, 'weight'):
        weights_0 = module.weight.data
    if hasattr(module, 'bias'):
        bias_0 = module.bias.data

theta_0 = torch.cat((weights_0[0], bias_0), 0)
theta_0

Training [lambda=0.0000; delta=0.5000]: 100%|██████████| 100/100 [00:03<00:00, 25.92epochs/s]


tensor([ 0.4047, -0.3873,  0.1086, -0.0497, -0.0455,  0.2492,  0.0892, -0.0266,
         0.0283,  0.2969, -0.0398, -0.2192, -0.0503, -0.0040, -0.0189,  0.0487,
         0.0349,  0.3141,  0.3925, -0.0152,  0.0889])

In [16]:
n = 21
alpha = .1
# thetas = torch.from_numpy(generate_grid(theta_0.numpy(), alpha, n=n, ord=np.inf)).float()
# thetas.shape

In [ ]:
Y_hat = model(test_dataset.X).reshape(-1).detach().numpy()
X = test_dataset.X[(Y_hat<tau),:]
Z = test_dataset.Z[(Y_hat<tau)]
X_hat_max = effort_model(model, test_dataset, X)

losses = []
loss_fn = torch.nn.BCELoss(reduction = 'mean')

for theta in tqdm(thetas):
    weights, bias = theta[:-1].clone().reshape(1, -1), theta[[-1]].clone()

    model_adv = deepcopy(model)
    for module in model_adv.layers:
        if hasattr(module, 'weight'):
            module.weight.data = weights.float()
        if hasattr(module, 'bias'):
            module.bias.data = bias.float()
            
    Y_hat_max = model_adv(X_hat_max).reshape(-1)
    
    fair_loss = 0.
    # Covariance Proxy
    if isinstance(ei_model, Covariance):
        fair_loss = torch.square(torch.mean((Z-Z.mean())*Y_hat_max))
    # Loss-based Proxy
    elif isinstance(ei_model, FairBatch):
        loss_mean = loss_fn(Y_hat_max, torch.ones(len(Y_hat_max)))
        loss_z = torch.zeros(len(dataset.sensitive_attrs))
        for z in test_dataset.sensitive_attrs:
            z = int(z)
            group_idx = (Z == z)
            if group_idx.sum() == 0:
                continue
            loss_z[z] = loss_fn(Y_hat_max[group_idx], torch.ones(group_idx.sum()))
            fair_loss += torch.abs(loss_z[z] - loss_mean)
    
    losses.append(fair_loss.detach().item())

In [ ]:
max_i = np.argmax(losses)
fair_loss = losses[max_i]
theta_r = thetas[max_i]
weights, bias = theta_r[:-1].clone().reshape(1, -1), theta_r[[-1]].clone()

model_adv = deepcopy(model)
for module in model_adv.layers:
    if hasattr(module, 'weight'):
        module.weight.data = weights.float()
    if hasattr(module, 'bias'):
        module.bias.data = bias.float()
        
X_hat_max = effort_model(model, test_dataset, test_dataset.X)
Y_hat_max = model_adv(X_hat_max).reshape(-1).detach().numpy()

accuracy, ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)

In [ ]:
alphas = (theta_r-theta_0).abs()
print(f'Grid Search Results {"(Covariance)" if isinstance(ei_model, Covariance) else "(Loss-based)"}')
print(f'alpha               |   {alpha}')
print(f'Accuracy            |   {accuracy:.5f}')
print(f'Fairness Loss       |   {fair_loss:.5f}')
print(f'EI Disparity        |   {ei_disparity:.5f}')
print(f'theta_0             |   {theta_0}')
print(f'theta_r             |   {theta_r}')
print(f'alphas              |   {np.round(alphas,4)}')

In [12]:
alpha = .1
pga_Y_hat, pga_Y_hat_max, pga_fair_loss = ei_model.predict(test_dataset, alpha, 1e-7)
pga_accuracy, pga_ei_disparity = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), pga_Y_hat, pga_Y_hat_max, tau)
for module in ei_model.model_adv.layers:
    if hasattr(module, 'weight'):
        pga_weights_r = module.weight.data
    if hasattr(module, 'bias'):
        pga_bias_r = module.bias.data
pga_theta_r = torch.cat((pga_weights_r[0], pga_bias_r), 0)

In [15]:
alphas = (pga_theta_r-theta_0).abs()
print(f'PGA Results {"(Covariance)" if isinstance(ei_model, Covariance) else "(Loss-based)"}')
print(f'alpha               |   {alpha}')
print(f'Accuracy            |   {pga_accuracy:.5f}')
print(f'Fairness Loss       |   {pga_fair_loss:.5f}')
print(f'EI Disparity        |   {pga_ei_disparity:.5f}')
# print(f'theta_0             |   {np.round(theta_0, 4)}')
# print(f'theta_r             |   {np.round(pga_theta_r, 4)}')
print(f'alphas              |   {np.round(alphas,4)}')

PGA Results (Covariance)
alpha               |   0.5
Accuracy            |   0.72500
Fairness Loss       |   0.01028
EI Disparity        |   0.40793
alphas              |   tensor([0.2978, 0.3895, 0.5000, 0.1801, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.0000, 0.2580, 0.5000, 0.3411, 0.5000, 0.3209, 0.5000, 0.2774, 0.5000,
        0.0000, 0.5000, 0.5000])
